In [43]:
import os
import time
import re
import json
from PIL import Image
import requests
import urllib.request
from bs4 import BeautifulSoup as bs
from selenium import webdriver

In [56]:
# path = "/root/chrome_driver/chromedriver"
path_window = r"/Users/seungsu/Desktop/YBIGTA/chromedriver.exe"
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--window-size=2400,1080")
# chrome_options.add_argument("headless")

driver = webdriver.Chrome(path_window, options=chrome_options)
driver.implicitly_wait(3)

#### 아이템 태그 리스트 얻기

In [27]:
p=2
test_url = category_url_list["knit"]["url"][:63] + f"page={p}&" + category_url_list["knit"]["url"][63:]
driver.get(test_url)
items = []
time.sleep(2)
ul = driver.find_element_by_css_selector("ul._53c8ee")
all_items = ul.find_elements_by_css_selector("li._c29d78._d85b45")
for item in all_items:
    box = item.find_element_by_css_selector("div._8eb9f8._a57b85")
    
    item_url = item.find_element_by_css_selector("a").get_attribute("href")
#     img = item.find_element_by_css_selector("img._fc8ffc").get_attribute("src")
    item_brand = box.find_element_by_css_selector("h3._346238").text
    item_name = box.find_element_by_css_selector("p._d85b45").text
    
    items.append({"brand":item_brand, "name":item_name, "href":item_url})
items

WebDriverException: Message: chrome not reachable
  (Session info: chrome=81.0.4044.138)


In [5]:
for i in items:
    print(i)

{'brand': 'RICK OWENS DRKSHDW', 'name': '크루 넥 스웨트셔츠', 'href': 'https://www.farfetch.com/kr/shopping/men/rick-owens-drkshdw--item-14360947.aspx?storeid=9359'}
{'brand': 'GUCCI', 'name': '아이콘 자카드 카디건', 'href': 'https://www.farfetch.com/kr/shopping/men/gucci--item-14835233.aspx?storeid=9359'}
{'brand': 'Z ZEGNA', 'name': '와플 니트 스웨터', 'href': 'https://www.farfetch.com/kr/shopping/men/z-zegna--item-14459481.aspx?storeid=9359'}
{'brand': 'RICK OWENS', 'name': '오버사이즈 집업 후디', 'href': 'https://www.farfetch.com/kr/shopping/men/rick-owens--item-14184047.aspx?storeid=9359'}
{'brand': 'GREG LAUREN', 'name': '텍스처 스웨터', 'href': 'https://www.farfetch.com/kr/shopping/men/greg-lauren--item-14184708.aspx?storeid=9359'}
{'brand': 'GREG LAUREN', 'name': 'X 폴앤샥 데님 앤 니트 후디', 'href': 'https://www.farfetch.com/kr/shopping/men/greg-lauren-x-item-14184701.aspx?storeid=9359'}
{'brand': 'GUCCI', 'name': '매니페스토 프린트 후디', 'href': 'https://www.farfetch.com/kr/shopping/men/gucci--item-14185485.aspx?storeid=9359'}
{'bra

#### 세부 데이터 URL

In [26]:
item_detail_url = items[4]["href"]
item_detail_url

'https://www.farfetch.com/kr/shopping/men/gucci--item-14835201.aspx?storeid=9359'

#### 세부 데이터 크롤링

In [21]:
# 데이터 저장 dict
detail_item_info = {}

# 접속
driver.get(item_detail_url)
time.sleep(2)
soup = bs(driver.page_source, 'html.parser')
print(soup)
# 기본정보
detail_item = driver.find_element_by_css_selector("div._9d3f24._da3196")
basic_info = driver.find_element_by_css_selector("div._c40757")
img_list = driver.find_element_by_css_selector("div._cc7da7").find_elements_by_css_selector("img._f3a89c")
category_box = driver.find_element_by_css_selector("ol._4e9dce")

detail_item_info["shop"] = "farfetch"
detail_item_info["brand"] = basic_info.find_element_by_css_selector("span._e87472._346238._e4b5ec").text
detail_item_info["product"] = basic_info.find_element_by_css_selector("span._d85b45._d85b45._1851d6").text
detail_item_info["product_id"] = basic_info.find_element_by_css_selector("span._d85b45._d85b45._1851d6").get_attribute("data-attribute")
detail_item_info["url"] = item_detail_url
detail_item_info["sub_category"] = category_box.find_elements_by_css_selector("li._9d7c74")[-1].find_element_by_css_selector("span").text
# detail_item_info["category"] = 
# detail_item_info["id"] = 
print(img_list)
# 이미지url
detail_item_img = []
for img in img_list:
    detail_item_img.append(img.get_attribute("src"))
detail_item_info["img"] = detail_item_img

detail_item_info
# 상세정보
# infos = driver.find_element_by_css_selector("div._9d3f24._da3196").find_elements_by_css_selector("div._4919a3._da3196")
# for row in infos:
#     pass

<html class=" js flexbox no-touch cssanimations csstransitions video svg placeholder haspointerevents csstransitionend not-ie win chrome chrome81 no-mobile-device win-chrome81" dir="LTR" lang="ko" style="--vh:949px;"><head> <title>Gucci 부티크 로고 프린트 후디 - Farfetch</title> <meta content="쇼핑 Gucci 부티크 로고 프린트 후디" name="description"/> <meta content="7jRAROVKvQYssFeSePEX2e9IQoHjq84OOvlu6i4hKN0" name="google-site-verification"/> <link href="/kr/shopping/men/gucci--item-14835201.aspx" rel="canonical"/> <link href="https://www.farfetch.com/shopping/men/gucci--item-14835201.aspx" hreflang="en-US" rel="alternate"/><link href="https://www.farfetch.com/ae/shopping/men/gucci--item-14835201.aspx" hreflang="en-AE" rel="alternate"/><link href="https://www.farfetch.com/ao/shopping/men/gucci--item-14835201.aspx" hreflang="en-AO" rel="alternate"/><link href="https://www.farfetch.com/ar/shopping/men/gucci--item-14835201.aspx" hreflang="en-AR" rel="alternate"/><link href="https://www.farfetch.com/ar-ae/shoppi

[]


{'shop': 'farfetch',
 'brand': 'GUCCI',
 'product': '부티크 로고 프린트 후디',
 'product_id': '14835201',
 'url': 'https://www.farfetch.com/kr/shopping/men/gucci--item-14835201.aspx?storeid=9359',
 'sub_category': '후디',
 'img': []}


# 개별 함수

### farfetch URL 만들기!!

In [52]:
category_url_list = {"knit" : {"category" : "knit", "url" : "https://www.farfetch.com/kr/shopping/men/clothing-2/items.aspx?view=180&category=136334"},
                    "denim" : {"category" : "denim", "url" : "https://www.farfetch.com/kr/shopping/men/clothing-2/items.aspx?view=180&category=136337"},
                    "shirts" : {"category" : "shirts", "url" : "https://www.farfetch.com/kr/shopping/men/clothing-2/items.aspx?view=180&category=136331"}, 
                    "shorts" : {"category" : "shorts", "url" : "https://www.farfetch.com/kr/shopping/men/clothing-2/items.aspx?view=180&category=136339"}, 
                    "suit" : {"category" : "suit", "url" : "https://www.farfetch.com/kr/shopping/men/clothing-2/items.aspx?view=180&category=136340"},
                    "jacket" : {"category" : "jacket", "url" : "https://www.farfetch.com/kr/shopping/men/clothing-2/items.aspx?view=180&category=136335"},
                    "coat" : {"category" : "coat", "url" : "https://www.farfetch.com/kr/shopping/men/clothing-2/items.aspx?view=180&category=136336"},
                    "t-shirt" : {"category" : "t-shirt", "url" : "https://www.farfetch.com/kr/shopping/men/clothing-2/items.aspx?view=180&category=136332"},
                    "pants" : {"category" : "pants", "url" : "https://www.farfetch.com/kr/shopping/men/clothing-2/items.aspx?view=180&category=136338"},
                    "polo-shirt" : {"category" : "polo-shirt", "url" : "https://www.farfetch.com/kr/shopping/men/clothing-2/items.aspx?view=180&category=136333"}
                    }

### 아이템리스트(by 페이지, 카테고리) 반환 함수

In [32]:
def getItemList(sub_category, page= 1):
#     self.category_url_list
    itemList_url = category_url_list[sub_category]["url"][:63] + f"page={str(page)}&" + category_url_list[sub_category]["url"][63:]
    driver.get(itemList_url)
    items = []
    time.sleep(2)
    ul = driver.find_element_by_css_selector("ul._53c8ee")
    all_items = ul.find_elements_by_css_selector("li._c29d78._d85b45")
    for item in all_items:
        box = item.find_element_by_css_selector("div._8eb9f8._a57b85")

        item_url = item.find_element_by_css_selector("a").get_attribute("href")
    #     img = item.find_element_by_css_selector("img._fc8ffc").get_attribute("src")
        item_brand = box.find_element_by_css_selector("h3._346238").text
        item_name = box.find_element_by_css_selector("p._d85b45").text

        items.append({"brand":item_brand, "name":item_name, "href":item_url})
    items
    return items

In [37]:
a = ['knit',
 'denim',
 'shirts',
 'shorts',
 'suit',
 'jacket',
 'coat',
 't-shirt',
 'pants',
 'polo-shirt']
# getItemList("knit", 1)

### 단일페이지 크롤러

In [45]:
def crawl_item(item_detail_url):
    # 데이터 저장 dict
    detail_item_info = {}

    # 접속
    driver.get(item_detail_url)
    time.sleep(2)
    soup = bs(driver.page_source, 'html.parser')
    
    # 기본정보
    detail_item = driver.find_element_by_css_selector("div._9d3f24._da3196")
    basic_info = driver.find_element_by_css_selector("div._c40757")
    img_list = driver.find_element_by_css_selector("div._cc7da7").find_elements_by_css_selector("img._f3a89c")
    category_box = driver.find_element_by_css_selector("ol._4e9dce")

    detail_item_info["shop"] = "farfetch"
    detail_item_info["brand"] = basic_info.find_element_by_css_selector("span._e87472._346238._e4b5ec").text
    detail_item_info["product"] = basic_info.find_element_by_css_selector("span._d85b45._d85b45._1851d6").text
    detail_item_info["product_id"] = basic_info.find_element_by_css_selector("span._d85b45._d85b45._1851d6").get_attribute("data-attribute")
    detail_item_info["url"] = item_detail_url
    detail_item_info["sub_category"] = category_box.find_elements_by_css_selector("li._9d7c74")[-1].find_element_by_css_selector("span").text
    # detail_item_info["category"] = 
    # detail_item_info["id"] = 
    
    # 이미지url
    detail_item_img = []
    for img in img_list:
        detail_item_img.append(img.get_attribute("src"))
    detail_item_info["img"] = detail_item_img

    detail_item_info
    # 상세정보
    # infos = driver.find_element_by_css_selector("div._9d3f24._da3196").find_elements_by_css_selector("div._4919a3._da3196")
    # for row in infos:
    #     pass    
    return detail_item_info

In [46]:
crawl_item(item_detail_url)

{'shop': 'farfetch',
 'brand': 'GUCCI',
 'product': '부티크 로고 프린트 후디',
 'product_id': '14835201',
 'url': 'https://www.farfetch.com/kr/shopping/men/gucci--item-14835201.aspx?storeid=9359',
 'sub_category': '후디',
 'img': ['https://cdn-images.farfetch-contents.com/14/83/52/01/14835201_26568780_1000.jpg',
  'https://cdn-images.farfetch-contents.com/14/83/52/01/14835201_26568779_1000.jpg',
  'https://cdn-images.farfetch-contents.com/14/83/52/01/14835201_26568782_1000.jpg',
  'https://cdn-images.farfetch-contents.com/14/83/52/01/14835201_26568781_1000.jpg',
  'https://cdn-images.farfetch-contents.com/14/83/52/01/14835201_26568783_1000.jpg']}

# 병합

### 고유 id 생성코드

In [47]:
def make_id(num,shop="13", category="00"):
#     10 : 무신사
#     11 : 29cm
#     12 : oco
#     13 : farfetch
#     14 : matchesfashion
    item_id = shop + category + str(num).rjust(5, "0")
    return item_id

### 한페이지 모든 데이터 크롤링 함수

In [48]:
# 아이템 dict을 저장할 dict
item_group = dict()
num = 1
for item in items:
    item_id = make_id(num, "13", "01")
    item_group[item_id] = crawl_item(item["href"])
    num += 1
item_group

KeyboardInterrupt: 

In [53]:
item_group

{'130100001': {'shop': 'farfetch',
  'brand': 'KENZO',
  'product': '파리 로고 인타르시아 니트 후디',
  'product_id': '13901899',
  'url': 'https://www.farfetch.com/kr/shopping/men/kenzo--item-13901899.aspx?storeid=9359',
  'sub_category': '후디',
  'img': ['https://cdn-images.farfetch-contents.com/13/90/18/99/13901899_22001820_1000.jpg',
   'https://cdn-images.farfetch-contents.com/13/90/18/99/13901899_22001825_1000.jpg',
   'https://cdn-images.farfetch-contents.com/13/90/18/99/13901899_22001830_1000.jpg',
   'https://cdn-images.farfetch-contents.com/13/90/18/99/13901899_22001835_1000.jpg',
   'https://cdn-images.farfetch-contents.com/13/90/18/99/13901899_22001838_1000.jpg']},
 '130100002': {'shop': 'farfetch',
  'brand': 'STONE ISLAND',
  'product': '고스트 하프 후드 집업',
  'product_id': '15332629',
  'url': 'https://www.farfetch.com/kr/shopping/men/stone-island--item-15332629.aspx?storeid=9359',
  'sub_category': '후디',
  'img': ['https://cdn-images.farfetch-contents.com/15/33/26/29/15332629_27139344_1000

### 카테고리 크롤링 함수

In [57]:
def crawl_category(sub_category, page, path=None ,save=False):
    # 아이템 dict을 저장할 dict
    item_group = dict()
    num = 1
    
    # page 별
    for p in range(1, int(page + 1)):
        # 페이지 item리스트 뽑기
        itemList = getItemList(sub_category, p)
        
        # 개별 아이템 json화
        for item in itemList:
            item_id = make_id(num, "13", "01")
            a = crawl_item(item["href"])
            item_group[item_id] = a
            print(a)
            num += 1
    
    # json으로 저장
    if save == True:
        # path 설정
        if path == None:
            path = f"./itemList.json"
        else:
            path = path + "/itemList.json"
        with open(path, 'w', encoding='utf-8') as make_file:
            json.dump(item_group, make_file, ensure_ascii=False, indent="\t")
    
    return item_group

In [58]:
crawl_category("knit", page=1, save=True)

{'shop': 'farfetch', 'brand': 'STONE ISLAND', 'product': '패널 크루 넥 스웨트셔츠', 'product_id': '14707672', 'url': 'https://www.farfetch.com/kr/shopping/men/stone-island--item-14707672.aspx?storeid=9359', 'sub_category': '스웨트셔츠', 'img': ['https://cdn-images.farfetch-contents.com/14/70/76/72/14707672_26623625_1000.jpg', 'https://cdn-images.farfetch-contents.com/14/70/76/72/14707672_26623624_1000.jpg', 'https://cdn-images.farfetch-contents.com/14/70/76/72/14707672_26631318_1000.jpg', 'https://cdn-images.farfetch-contents.com/14/70/76/72/14707672_26623627_1000.jpg', 'https://cdn-images.farfetch-contents.com/14/70/76/72/14707672_26623628_1000.jpg']}
{'shop': 'farfetch', 'brand': 'PALM ANGELS', 'product': '로고 자수 스웨트셔츠', 'product_id': '15359671', 'url': 'https://www.farfetch.com/kr/shopping/men/palm-angels--item-15359671.aspx?storeid=9359', 'sub_category': '스웨트셔츠', 'img': []}
{'shop': 'farfetch', 'brand': 'AMIRI', 'product': '로고 크루 넥 스웨트셔츠', 'product_id': '14122643', 'url': 'https://www.farfetch.com

{'shop': 'farfetch', 'brand': 'KENZO', 'product': '클래식 타이거 로고 스웨터', 'product_id': '14502862', 'url': 'https://www.farfetch.com/kr/shopping/men/kenzo--item-14502862.aspx?storeid=9359', 'sub_category': '스웨트셔츠', 'img': []}
{'shop': 'farfetch', 'brand': 'Y-3', 'product': '로고 디테일 크루 스웨터', 'product_id': '14747755', 'url': 'https://www.farfetch.com/kr/shopping/men/y-3--item-14747755.aspx?storeid=9359', 'sub_category': '니트웨어', 'img': []}
{'shop': 'farfetch', 'brand': 'POLO RALPH LAUREN', 'product': '테디 프린트 로고 후디', 'product_id': '14539918', 'url': 'https://www.farfetch.com/kr/shopping/men/polo-ralph-lauren--item-14539918.aspx?storeid=9359', 'sub_category': '후디', 'img': []}
{'shop': 'farfetch', 'brand': 'BILLIONAIRE BOYS CLUB', 'product': '믹스 프린트 스웨트셔츠', 'product_id': '15360132', 'url': 'https://www.farfetch.com/kr/shopping/men/billionaire-boys-club--item-15360132.aspx?storeid=9359', 'sub_category': '스웨트셔츠', 'img': []}
{'shop': 'farfetch', 'brand': 'CHAMPION', 'product': '드로스트링 긴소매 후디', 'product

{'shop': 'farfetch', 'brand': 'CHAMPION', 'product': '크루 넥 스웨트셔츠', 'product_id': '14432048', 'url': 'https://www.farfetch.com/kr/shopping/men/champion--item-14432048.aspx?storeid=9359', 'sub_category': '스웨트셔츠', 'img': ['https://cdn-images.farfetch-contents.com/14/43/20/48/14432048_24878195_1000.jpg', 'https://cdn-images.farfetch-contents.com/14/43/20/48/14432048_24878194_1000.jpg', 'https://cdn-images.farfetch-contents.com/14/43/20/48/14432048_24878198_1000.jpg', 'https://cdn-images.farfetch-contents.com/14/43/20/48/14432048_24878201_1000.jpg', 'https://cdn-images.farfetch-contents.com/14/43/20/48/14432048_24878202_1000.jpg']}
{'shop': 'farfetch', 'brand': 'GUCCI', 'product': '스트라이프 롤 넥 스웨터', 'product_id': '14880704', 'url': 'https://www.farfetch.com/kr/shopping/men/gucci--item-14880704.aspx?storeid=9359', 'sub_category': '스웨터', 'img': ['https://cdn-images.farfetch-contents.com/14/88/07/04/14880704_26682274_1000.jpg', 'https://cdn-images.farfetch-contents.com/14/88/07/04/14880704_26682

{'shop': 'farfetch', 'brand': 'BURBERRY', 'product': '아트 프린트 후디', 'product_id': '15012010', 'url': 'https://www.farfetch.com/kr/shopping/men/burberry--item-15012010.aspx?storeid=9359', 'sub_category': '후디', 'img': ['https://cdn-images.farfetch-contents.com/15/01/20/10/15012010_25514654_1000.jpg', 'https://cdn-images.farfetch-contents.com/15/01/20/10/15012010_25514656_1000.jpg', 'https://cdn-images.farfetch-contents.com/15/01/20/10/15012010_25514657_480.jpg', 'https://cdn-images.farfetch-contents.com/15/01/20/10/15012010_25514658_322.jpg', 'https://cdn-images.farfetch-contents.com/15/01/20/10/15012010_25514660_322.jpg', 'https://cdn-images.farfetch-contents.com/15/01/20/10/15012010_898506e7-38e1-4d57-b436-ca286863baa7_1000.jpg']}
{'shop': 'farfetch', 'brand': 'STONE ISLAND', 'product': '고스트 하프 후드 집업', 'product_id': '15335871', 'url': 'https://www.farfetch.com/kr/shopping/men/stone-island--item-15335871.aspx?storeid=9359', 'sub_category': '후디', 'img': ['https://cdn-images.farfetch-conten

{'shop': 'farfetch', 'brand': 'GIVENCHY', 'product': '컬러 블록 집업 스웨트셔츠', 'product_id': '14143927', 'url': 'https://www.farfetch.com/kr/shopping/men/givenchy--item-14143927.aspx?storeid=9359', 'sub_category': '스웨트셔츠', 'img': ['https://cdn-images.farfetch-contents.com/14/14/39/27/14143927_26212817_1000.jpg', 'https://cdn-images.farfetch-contents.com/14/14/39/27/14143927_26212815_1000.jpg', 'https://cdn-images.farfetch-contents.com/14/14/39/27/14143927_26212818_1000.jpg', 'https://cdn-images.farfetch-contents.com/14/14/39/27/14143927_26212819_1000.jpg', 'https://cdn-images.farfetch-contents.com/14/14/39/27/14143927_26212822_480.jpg']}
{'shop': 'farfetch', 'brand': 'ALEXANDER MCQUEEN', 'product': '드래곤 로고 스웨트셔츠', 'product_id': '14528102', 'url': 'https://www.farfetch.com/kr/shopping/men/alexander-mcqueen--item-14528102.aspx?storeid=9359', 'sub_category': '스웨터', 'img': ['https://cdn-images.farfetch-contents.com/14/52/81/02/14528102_25348893_1000.jpg', 'https://cdn-images.farfetch-contents.com/

...

{'shop': 'farfetch', 'brand': 'CHAMPION', 'product': '로고 드로스트링 후디', 'product_id': '14432029', 'url': 'https://www.farfetch.com/kr/shopping/men/champion--item-14432029.aspx?storeid=9359', 'sub_category': '니트웨어', 'img': ['https://cdn-images.farfetch-contents.com/14/43/20/29/14432029_24878186_1000.jpg', 'https://cdn-images.farfetch-contents.com/14/43/20/29/14432029_24878189_1000.jpg', 'https://cdn-images.farfetch-contents.com/14/43/20/29/14432029_24878191_1000.jpg', 'https://cdn-images.farfetch-contents.com/14/43/20/29/14432029_24878193_1000.jpg', 'https://cdn-images.farfetch-contents.com/14/43/20/29/14432029_24878192_480.jpg']}
{'shop': 'farfetch', 'brand': 'POLO RALPH LAUREN', 'product': '모티프 자수 스웨트셔츠', 'product_id': '14143695', 'url': 'https://www.farfetch.com/kr/shopping/men/polo-ralph-lauren--item-14143695.aspx?storeid=9359', 'sub_category': '스웨트셔츠', 'img': ['https://cdn-images.farfetch-contents.com/14/14/36/95/14143695_20398012_1000.jpg', 'https://cdn-images.farfetch-contents.com/14

{'shop': 'farfetch', 'brand': 'ALEXANDER MCQUEEN', 'product': '패널 로고 프린트 후디', 'product_id': '14528104', 'url': 'https://www.farfetch.com/kr/shopping/men/alexander-mcqueen--item-14528104.aspx?storeid=9359', 'sub_category': '후디', 'img': ['https://cdn-images.farfetch-contents.com/14/52/81/04/14528104_23187929_1000.jpg', 'https://cdn-images.farfetch-contents.com/14/52/81/04/14528104_23187930_480.jpg', 'https://cdn-images.farfetch-contents.com/14/52/81/04/14528104_23187932_1000.jpg', 'https://cdn-images.farfetch-contents.com/14/52/81/04/14528104_23187935_322.jpg', 'https://cdn-images.farfetch-contents.com/14/52/81/04/14528104_23187936_480.jpg']}
{'shop': 'farfetch', 'brand': 'CANADA GOOSE', 'product': '윈드브릿지 패널 후디', 'product_id': '14850052', 'url': 'https://www.farfetch.com/kr/shopping/men/canada-goose--item-14850052.aspx?storeid=9359', 'sub_category': '후디', 'img': []}
{'shop': 'farfetch', 'brand': 'RICK OWENS X CHAMPION', 'product': '릭 오웬스 x 챔피언 로고 자수 코튼 혼방 스웨트셔츠', 'product_id': '14354766'

{'shop': 'farfetch', 'brand': 'SAINT LAURENT', 'product': '케이블 니트 후드 카디건', 'product_id': '13813037', 'url': 'https://www.farfetch.com/kr/shopping/men/saint-laurent--item-13813037.aspx?storeid=9359', 'sub_category': '카디건', 'img': ['https://cdn-images.farfetch-contents.com/13/81/30/37/13813037_17300013_480.jpg', 'https://cdn-images.farfetch-contents.com/13/81/30/37/13813037_17300015_1000.jpg', 'https://cdn-images.farfetch-contents.com/13/81/30/37/13813037_17300017_1000.jpg', 'https://cdn-images.farfetch-contents.com/13/81/30/37/13813037_17300019_322.jpg', 'https://cdn-images.farfetch-contents.com/13/81/30/37/13813037_17300021_480.jpg']}
{'shop': 'farfetch', 'brand': 'BALMAIN', 'product': '크루 넥 울 인타르시아 스웨터', 'product_id': '14020262', 'url': 'https://www.farfetch.com/kr/shopping/men/balmain--item-14020262.aspx?storeid=9359', 'sub_category': '스웨트셔츠', 'img': ['https://cdn-images.farfetch-contents.com/14/02/02/62/14020262_19046196_1000.jpg', 'https://cdn-images.farfetch-contents.com/14/02/02/

{'shop': 'farfetch', 'brand': 'BALMAIN', 'product': '버튼 디테일 로고 인타르시아 스웨터', 'product_id': '15340134', 'url': 'https://www.farfetch.com/kr/shopping/men/balmain--item-15340134.aspx?storeid=9359', 'sub_category': '스웨터', 'img': ['https://cdn-images.farfetch-contents.com/15/34/01/34/15340134_27232863_1000.jpg', 'https://cdn-images.farfetch-contents.com/15/34/01/34/15340134_27232862_1000.jpg']}
{'shop': 'farfetch', 'brand': 'GUCCI', 'product': 'GG 로고 니트 판초', 'product_id': '13264160', 'url': 'https://www.farfetch.com/kr/shopping/men/gucci-gg-item-13264160.aspx?storeid=9359', 'sub_category': '스웨트셔츠', 'img': ['https://cdn-images.farfetch-contents.com/13/26/41/60/13264160_14782011_480.jpg', 'https://cdn-images.farfetch-contents.com/13/26/41/60/13264160_14782035_480.jpg', 'https://cdn-images.farfetch-contents.com/13/26/41/60/13264160_14782051_480.jpg', 'https://cdn-images.farfetch-contents.com/13/26/41/60/13264160_14782067_322.jpg', 'https://cdn-images.farfetch-contents.com/13/26/41/60/13264160_14

{'130100001': {'shop': 'farfetch',
  'brand': 'STONE ISLAND',
  'product': '패널 크루 넥 스웨트셔츠',
  'product_id': '14707672',
  'url': 'https://www.farfetch.com/kr/shopping/men/stone-island--item-14707672.aspx?storeid=9359',
  'sub_category': '스웨트셔츠',
  'img': ['https://cdn-images.farfetch-contents.com/14/70/76/72/14707672_26623625_1000.jpg',
   'https://cdn-images.farfetch-contents.com/14/70/76/72/14707672_26623624_1000.jpg',
   'https://cdn-images.farfetch-contents.com/14/70/76/72/14707672_26631318_1000.jpg',
   'https://cdn-images.farfetch-contents.com/14/70/76/72/14707672_26623627_1000.jpg',
   'https://cdn-images.farfetch-contents.com/14/70/76/72/14707672_26623628_1000.jpg']},
 '130100002': {'shop': 'farfetch',
  'brand': 'PALM ANGELS',
  'product': '로고 자수 스웨트셔츠',
  'product_id': '15359671',
  'url': 'https://www.farfetch.com/kr/shopping/men/palm-angels--item-15359671.aspx?storeid=9359',
  'sub_category': '스웨트셔츠',
  'img': []},
 '130100003': {'shop': 'farfetch',
  'brand': 'AMIRI',
  'p

### 저장용 디렉토리 생성

In [5]:
def check_directory(category_name, folder_path=None):
    if folder_path == None:
        folder_path = "C:/Users/seungsu/Desktop/ICT-project/crawling"
    try:
        os.mkdir(f"{folder_path}/{category_name}/")
        print("Directory is created!")
    except:
        print("Directory exist!")